Pada sub-modul ini, kita akan belajar menjalankan model yang telah kita buat untuk melakukan tugas prediksi dalam lingkup pemrosesan bahasa alami (Natural Language Processing). Tugas prediksi tersebut akan dijalankan pada Web Browser menggunakan TensorFlow.js. Adapun pengetahuan tambahan tentang HTML, CSS, JavaScript, dan Chrome Dev Tools dapat membantu Anda memahami sub-modul ini dengan baik.

Berikut beberapa hal yang akan kita pelajari:

Memuat dan menjalankan model pada Web Browser dengan TensorFlow.js API.
Menguji hasil deployment dengan membuat prediksi dari input yang diberikan.
Pada latihan ini, kita akan membuat laman web yang dapat memprediksi sentimen dari sebuah reviu restoran. Model yang digunakan akan dilatih menggunakan dataset reviu restoran Yelp. 

Anda dapat mengunduh contoh proyek pada tautan berikut: contoh proyek. 

Tugas Anda adalah melengkapi bagian yang kosong sesuai instruksi yang diberikan.

# **Kebutuhan Sistem:**
Dalam latihan kali ini, Anda akan menjalankan laman web secara lokal di komputer menggunakan web server lokal. Perangkat lunak yang dibutuhkan, antara lain:

* Browser internet versi terbaru (Chrome)
* Text editor (Sublime Text, Bracker, VS Code, dan lain-lain)
* Web server yang ter-install di komputer (Web Server for Chrome)

Jika Anda belum menginstal Web Server for Chrome, Anda dapat mengikuti langkah berikut. Pertama, tambahkan “Web Server for Chrome” sebagai extensions pada browser Google Chrome Anda melalui tautan berikut: Web Server for Chrome. https://chrome.google.com/webstore/detail/web-server-for-chrome/ofhbbkphhbklhfoeikjpcbhemlocgigb?hl=en

Selanjutnya untuk mengaktifkan “Web Server for Chrome”, geser toggle hingga statusnya berubah menjadi “WebServer:STARTED”, kemudian pilih folder project. 

Salin http://127.0.0.1:8887 pada alamat browser, lalu buka file index.html. Berikut contoh tampilan “Web Server for Chrome”.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:b5c7c92590752f5a68f8ae445ea25a3b20211208092532.jpeg

# **Penjelasan Proyek**
Tujuan dari proyek ini adalah membuat sebuah model yang dapat memprediksi apakah sebuah restoran mendapatkan reviu positif atau negatif. Model ini nantinya akan dimuat dan dijalankan pada Web Browser dengan menggunakan TensorFlow.js API https://www.tensorflow.org/js.  

Berikut merupakan tampilan awal laman web dari proyek yang akan kita buat nanti.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:eb912f768d1f4f5c41bb8a116bd1653120211208092854.jpeg

Proyek ini menggunakan dataset yield https://www.kaggle.com/datasets/marklvl/sentiment-labelled-sentences-data-set. Dataset tersebut berisi kumpulan reviu terhadap sebuah restoran. Dataset ini terdiri dari 1000 sampel reviu yang dikelompokkan berdasarkan kategori reviu positif atau negatif.



# **Penjelasan File**
Setelah mengunduh contoh proyek, Anda akan menjumpai beberapa file dan folder sebagai berikut:

* `notebook.ipynb` merupakan berkas notebook yang dapat dijalankan melalui Google Colaboratory. Notebook ini berisi tahapan dalam membuat, mengevaluasi, dan mengonversi model.
* `yelp_labelled.txt` merupakan dataset untuk melatih model.
* `model.h5` merupakan sebuah HDF5 model yang akan dikonversi menjadi `model.json`.
* `tfjs_model` merupakan sebuah folder yang berisi `model.json` dan bobot dalam berbentuk binary file. Kedua file tersebut digunakan untuk menerapkan model NLP ke dalam web browser.
* `word_index.json` merupakan sebuah metadata dalam bentuk file json yang berisi pasangan kata dan indeks. Dengan memanfaatkan berkas ini, kita dapat mengubah inputan reviu menjadi sebuah token.
* `index.html` merupakan berkas html sebagai tampilan utama web.
* `script.js` merupakan berkas JavaScript yang berisi perintah untuk men-deploy model yang telah dibuat ke dalam web.
* `images` berisi beberapa gambar seperti plot evaluasi model yang telah dibuat dan tampilan awal web.
* `styles` berisi `style.css` dan beberapa gambar untuk memperindah tampilan web.

# **Penjelasan Notebook**
Bagian ini akan membahas tahap pembuatan model untuk mengklasifikasi reviu sebuah restoran sebagai positive review atau negative review. Sebelum membuat model, kita perlu mempersiapkan semua library yang dibutuhkan.

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
import json

Setelah semua library siap, tahap selanjutnya adalah menyiapkan dataset yang akan digunakan.

In [ ]:
df = pd.read_csv('yelp_labelled.txt', names=['sentence', 'label'], sep='\t')

Kemudian, lakukan tahap preprocessing data berikut:

* Mengubah seluruh text kedalam bentuk lowercase.
* Menghilangkan stopwords.
* Melakukan split dataset.
* Melakukan tokenisasi.
* Membuat sequences dan melakukan padding.

 Selain tahap preprocessing, kita juga perlu menyiapkan metadata dan menyimpannya dalam file word_index.json.

In [ ]:
# Mengubah seluruh text kedalam bentuk lowercase
df['sentence'] = df['sentence'].str.lower()
 
# Menghilangkan stopwords
stop_word = set(stopwords.words('english'))
 
df['sentence'] = df['sentence'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_word)]))
 
# Melakukan split dataset
sentence = df['sentence'].values
label = df['label'].values
 
sentence_train, sentence_test, label_train, label_test = train_test_split(sentence, label, test_size=0.2, shuffle=False)
 
# Membuat tokenisasi
filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' # Filter untuk menghilangkan symbols
 
tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>", filters=filt)
 
tokenizer.fit_on_texts(sentence_train)
 
# Menyimpan word_index kedalam sebuah file json
word_index = tokenizer.word_index
 
with open('word_index.json', 'w') as fp:
    json.dump(word_index, fp)
 
# Membuat sequences dan melakukan padding
train_sekuens = tokenizer.texts_to_sequences(sentence_train)
test_sekuens = tokenizer.texts_to_sequences(sentence_test)
 
train_padded = pad_sequences(train_sekuens,
                             maxlen=20,
                             padding='post',
                             truncating='post')
test_padded = pad_sequences(test_sekuens,
                            maxlen=20,
                            padding='post',
                            truncating='post')

Tahap berikutnya adalah membuat dan melatih model menggunakan dataset yang telah kita siapkan sebelumnya.

In [ ]:
# Membuat model
model = tf.keras.Sequential([
    Embedding(2000, 20, input_length=20),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
 
# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
 
# Train model
num_epochs = 30
history = model.fit(train_padded, label_train,
                    epochs=num_epochs,
                    validation_data=(test_padded, label_test),
                    verbose=1)

Jika performa model dirasa cukup, simpanlah dalam format HDF5 menggunakan perintah berikut:

In [ ]:
model.save("model.h5")

Nah, sebelum men-deploy model ke dalam web, ubahlah HDF5 model yang telah disimpan tadi ke dalam bentuk file json.

In [ ]:
# Install tensorflowjs
!pip install tensorflowjs
 
# Convert model.h5 to model
!tensorflowjs_converter --input_format=keras model.h5 tfjs_model

Jika semua tahapan berjalan dengan baik, akan muncul sebuah folder baru bernama `tfjs_model` yang berisi `model.json` & weight berbentuk binary file. Selamat! Model Anda telah siap untuk masuk ke tahap deployment dengan TensorFlow.js.

Yuk, kita lanjut ke tahapan berikutnya!

# **Menerapkan Model pada Web Browser dengan TensorFlow.js**
Untuk menerapkan model yang telah dibuat di web browser, buatlah file ‘index.html’ sebagai tampilan utama web. Selain itu, buatlah file ‘script.js’ yang berisi perintah untuk men-deploy model.

Pada contoh proyek yang telah diberikan, Anda akan menemukan file ‘index.html’ dan file ‘script.js’. Buka kedua file tersebut menggunakan text editor favorit Anda, kemudian ikuti petunjuk berikut.



**Membuat file ‘index.html’**

Setelah Anda membuka file ‘index.html’, tambahkan perintah berikut pada bagian tag <head> untuk menampilkan title dan memanggil ‘style.css’.

In [ ]:
 <title>Tensorflow.js: Sentiment Analysis Demo</title>
   
    <link href="styles/style.css" rel="stylesheet">
    <link href='https://fonts.googleapis.com/css?family=Sofia' rel='stylesheet'>      
    <link href='https://fonts.googleapis.com/css?family=Oswald' rel='stylesheet'>  
    <link href='https://fonts.googleapis.com/css?family=Pontano Sans' rel='stylesheet'>

Kemudian, pada tag yang sama, tambahkanlah perintah berikut untuk memanggil TensorFlow.js API versi terbaru. 

In [ ]:
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@latest"> </script>

Lalu, pada bagian tag <body> tambahkan perintah untuk membuat tampilan utama web seperti berikut.

In [ ]:
<div class = "header">
      <img src="styles/logo.jpg"></img>
    </div>
   
    <div class = "main">
    <div class="container">
      <img src="styles/tensorflow_logo.png"></img>
      <link rel="stylesheet" href="//fonts.googleapis.com/css?family=Open+Sans:300,400,600,700&amp;lang=en" />
      <link href='https://fonts.googleapis.com/css?family=Archivo Narrow' rel='stylesheet'>
     
      <h1>Tensorflow.js</h1>
      <h2>Sentiment Analysis Demo</h2>
     
      <h3>
      <p>
        This example demonstrated loading a pre-trained model and using it in the browser. This model is trained to
        predict the sentiment of a short restaurant review (as a score between 0 and 1). The training is done server side
        using Python and then converted into a TensorFlow.js model.
      </p>
      <p>
        The model is trained using YELP reviews that have been truncated to a maximum of 20 words, only the
        2000 most used words in the reviews are used. You can experiment with the model on this page
      </p>
      </h3>
     
    </div>
    </div>
   
    <div class="container">
 
    <div class = "demo">
      <h1>Thank you for your visit!</h1>
      <h2>We are committed to providing our customers with a fantastic experience, and your feedback helps us make that possible.</h2>
      <br><br>
      <div id="loaderlabel"<h2>Loading Model...</h2></div>
      <div id="loader"> </div>  
      <div style="display:none;" id="mainAPP">
      <label for="input">How was your dinner? </label><br><br>
      <input placeholder="Try: 'The salad is fresh and delicious" id="input">
      <button class="button" style="vertical-align:middle"><span>Post Review </span></button>
      <p></p>
    </div>    
    </div>  
    </div>

Sebagai penutup, tambahkan perintah berikut pada bagian tag <body> untuk memanggil ‘script.js’..

In [ ]:
 <script src="script.js"></script>

Setelah semua tahap dilakukan, file `index.html` Anda akan terlihat seperti ini:

In [ ]:
<html>
  <head>
    <title>Tensorflow.js: Sentiment Analysis Demo</title>
   
    <link href="styles/style.css" rel="stylesheet">
    <link href='https://fonts.googleapis.com/css?family=Sofia' rel='stylesheet'>      
    <link href='https://fonts.googleapis.com/css?family=Oswald' rel='stylesheet'>  
    <link href='https://fonts.googleapis.com/css?family=Pontano Sans' rel='stylesheet'>
   
    <!-- Memanggil versi terbaru TensorFlow.js -->
    <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@latest"> </script>
    <!--  -->
  </head>
  <body onload="myFunction()" style="margin:0;">
    <div class = "header">
      <img src="styles/logo.jpg"></img>
    </div>
   
    <!-- Add a <div> tag with the "main" class -->
    <div class = "main">
    <div class="container">
      <img src="styles/tensorflow_logo.png"></img>
      <link rel="stylesheet" href="//fonts.googleapis.com/css?family=Open+Sans:300,400,600,700&amp;lang=en" />
      <link href='https://fonts.googleapis.com/css?family=Archivo Narrow' rel='stylesheet'>
     
      <h1>Tensorflow.js</h1>
      <h2>Sentiment Analysis Demo</h2>
     
      <h3>
      <p>
        This example demonstrated loading a pre-trained model and using it in the browser. This model is trained to
        predict the sentiment of a short restaurant review (as a score between 0 and 1). The training is done server side
        using Python and then converted into a TensorFlow.js model.
      </p>
      <p>
        The model is trained using YELP reviews that have been truncated to a maximum of 20 words, only the
        2000 most used words in the reviews are used. You can experiment with the model on this page
      </p>
      </h3>
     
    </div>
    </div>
   
    <div class="container">
    <div class = "demo">
      <h1>Thank you for your visit!</h1>
      <h2>We are committed to providing our customers with a fantastic experience, and your feedback helps us make that possible.</h2>
      <br><br>
      <div id="loaderlabel"<h2>Loading Model...</h2></div>
      <div id="loader"> </div>  
      <div style="display:none;" id="mainAPP">
      <label for="input">How was your dinner? </label><br><br>
      <input placeholder="Try: 'The salad is fresh and delicious" id="input">
      <button class="button" style="vertical-align:middle"><span>Post Review </span></button>
      <p></p>
    </div>    
    </div>  
         
    </div>
      <!-- Memanggil script.js -->
      <script src="script.js"></script>
      <!--  -->
  </body>
</html>

**Membuat file ‘script.js’**

Pertama bukalah file ‘script.js’ menggunakan text editor. Selanjutnya deklarasikan beberapa variabel yang akan digunakan.

In [ ]:
let input = document.querySelector('input');
let button = document.querySelector('button');
button.addEventListener('click', onClick);
 
let isModelLoaded = false;
let model;
let word2index;
 
// Parameter data preprocessing
const maxlen = 20;
const vocab_size = 2000;
const padding = 'post';
const truncating = 'post';

Tahapan berikutnya adalah membuat fungsi getInput(). Fungsi ini digunakan untuk mengambil inputan reviu yang dimasukkan oleh user sebagai data baru.

In [ ]:
function getInput(){
    const reviewText = document.getElementById('input')
    return reviewText.value;
}

Setelah membuat fungsi getInput(), buatlah sebuah fungsi untuk melakukan preprocessing terhadap data baru yang diinput. Tahap preprocessing ini dilakukan dengan menambahkan fungsi padSequence().

In [ ]:
function padSequence(sequences, maxLen, padding='post', truncating = "post", pad_value = 0){
    return sequences.map(seq => {
        if (seq.length > maxLen) { //truncat
            if (truncating === 'pre'){
                seq.splice(0, seq.length - maxLen);
            } else {
                seq.splice(maxLen, seq.length - maxLen);
            }
        }
               
        if (seq.length < maxLen) {
            const pad = [];
            for (let i = 0; i < maxLen - seq.length; i++){
                pad.push(pad_value);
            }
            if (padding === 'pre') {
                seq = pad.concat(seq);
            } else {
                seq = seq.concat(pad);
            }
        }              
        return seq;
        });
}

Selanjutnya, buatlah fungsi predict() yang digunakan untuk melakukan prediksi. Namun, kita perlu melakukan preprocessing terhadap data input terlebih dahulu. Tahap preprocessing dilakukan dengan mengubah data input menjadi sebuah token (tokenisasi). Kemudian, lakukanlah padding dengan memanggil fungsi padSequence() yang sebelumnya telah dibuat. 

Perlu diingat, output dari fungsi ini berupa nilai probabilitas sebuah reviu yang diklasifikasikan sebagai Positive Review atau Negative Review. 

In [ ]:
function predict(inputText){
 
    // Mengubah input review ke dalam bentuk token
    const sequence = inputText.map(word => {
        let indexed = word2index[word];
 
        if (indexed === undefined){
            return 1; //change to oov value
        }
        return indexed;
    });
   
    // Melakukan padding
    const paddedSequence = padSequence([sequence], maxlen);
 
    const score = tf.tidy(() => {
        const input = tf.tensor2d(paddedSequence, [1, maxlen]);
        const result = model.predict(input);
        return result.dataSync()[0];
    });
 
    return score;  
 
}

Kemudian, buatlah sebuah fungsi onClick(). Fungsi ini akan dijalankan ketika tombol "Post Review" ditekan. Pada fungsi ini terdapat perintah untuk menjalankan proses prediksi. Selain itu, pada fungsi ini juga ada pernyataan kondisi (conditional statement) untuk menentukan hasil prediksi berdasarkan nilai "score".

In [ ]:
function onClick(){
   
    if(!isModelLoaded) {
        alert('Model not loaded yet');
        return;
    }
 
    if (getInput() === '') {
        alert("Review Can't be Null");
        document.getElementById('input').focus();
        return;
    }
   
    //
    const inputText = getInput().trim().toLowerCase().split(" ");
 
    // Score prediksi dengan nilai 0 s/d 1
    let score = predict(inputText);
 
    // Kondisi penentuan hasil prediksi berdasarkan nilai score
    if (score > 0.5) {
        alert ('Positive Review \n'+score);
    } else {
        alert ('Negative Review \n'+score);
    }
}

Sebagai penutup, buatlah fungsi init(). Pada fungsi tersebut, terdapat perintah untuk memanggil 'model.json' dan 'word_index.json' yang telah dibuat sebelumnya. Selain itu, pada fungsi ini juga terdapat perintah untuk menampilkan detail arsitektur model yang digunakan melalui console log.

In [ ]:
async function init(){
 
    // Memanggil model tfjs
    model = await tf.loadLayersModel('http://127.0.0.1:8887/tfjs_model/model.json');
    isModelLoaded = true;
 
    //Memanggil word_index
    const word_indexjson = await fetch('http://127.0.0.1:8887/word_index.json');
    word2index = await word_indexjson.json();
 
    console.log(model.summary());
    console.log('Model & Metadata Loaded Successfully');
}

Setelah semua tahap dilakukan, file 'script.js' Anda akan terlihat seperti ini:

In [ ]:
// ----Kolom deklarasi variabel-----
let input = document.querySelector('input');
let button = document.querySelector('button');
button.addEventListener('click', onClick);
 
let isModelLoaded = false;
let model;
let word2index;
 
// Parameter data preprocessing
const maxlen = 20;
const vocab_size = 2000;
const padding = 'post';
const truncating = 'post';
 
var myVar;
// -----------------------------------
 
function myFunction() {
    myVar = setTimeout(showPage, 3000);
}
 
function showPage() {
    document.getElementById("loaderlabel").style.display = "none";
    document.getElementById("loader").style.display = "none";      
    document.getElementById("mainAPP").style.display = "block";
}
 
function detectWebGLContext () {
    // Create canvas element. The canvas is not added to the
    // document itself, so it is never displayed in the
    // browser window.
    var canvas = document.createElement("canvas");
    // Get WebGLRenderingContext from canvas element.
    var gl = canvas.getContext("webgl") || canvas.getContext("experimental-webgl");
    // Report the result.
    if (gl && gl instanceof WebGLRenderingContext) {
        console.log("Congratulations! Your browser supports WebGL.");
        init();
    } else {
        alert("Failed to get WebGL context. Your browser or device may not support WebGL.");
    }
}
 
detectWebGLContext();
 
// ----Kolom fungsi `getInput()`-----
// Fungsi untuk mengambil input review
function getInput(){
    const reviewText = document.getElementById('input')
    return reviewText.value;
}
// -----------------------------------
 
// ----Kolom fungsi `padSequence()`-----
// Fungsi untuk melakukan padding
function padSequence(sequences, maxLen, padding='post', truncating = "post", pad_value = 0){
    return sequences.map(seq => {
        if (seq.length > maxLen) { //truncat
            if (truncating === 'pre'){
                seq.splice(0, seq.length - maxLen);
            } else {
                seq.splice(maxLen, seq.length - maxLen);
            }
        }
               
        if (seq.length < maxLen) {
            const pad = [];
            for (let i = 0; i < maxLen - seq.length; i++){
                pad.push(pad_value);
            }
            if (padding === 'pre') {
                seq = pad.concat(seq);
            } else {
                seq = seq.concat(pad);
            }
        }              
        return seq;
        });
}
// -----------------------------------
 
 
// ----Kolom fungsi `predict()`-----
// Fungsi untuk melakukan prediksi
function predict(inputText){
 
    // Mengubah input review ke dalam bentuk token
    const sequence = inputText.map(word => {
        let indexed = word2index[word];
 
        if (indexed === undefined){
            return 1; //change to oov value
        }
        return indexed;
    });
   
    // Melakukan padding
    const paddedSequence = padSequence([sequence], maxlen);
 
    const score = tf.tidy(() => {
        const input = tf.tensor2d(paddedSequence, [1, maxlen]);
        const result = model.predict(input);
        return result.dataSync()[0];
    });
 
    return score;  
 
}
// -----------------------------------
 
 
// ----Kolom fungsi `onClick()`-----
// Fungsi yang dijalankan ketika tombol "Post Review" diclick
function onClick(){
   
    if(!isModelLoaded) {
        alert('Model not loaded yet');
        return;
    }
 
    if (getInput() === '') {
        alert("Review Can't be Null");
        document.getElementById('input').focus();
        return;
    }
   
    //
    const inputText = getInput().trim().toLowerCase().split(" ");
 
    // Score prediksi dengan nilai 0 s/d 1
    let score = predict(inputText);
 
    // Kondisi penentuan hasil prediksi berdasarkan nilai score
    if (score > 0.5) {
        alert ('Positive Review \n'+score);
    } else {
        alert ('Negative Review \n'+score);
    }
}
// -----------------------------------
 
 
// ----Kolom fungsi `init()`-----
async function init(){
 
    // Memanggil model tfjs
    // model = await tf.loadLayersModel('http://127.0.0.1:5500/tfjs_model/model.json'); // Untuk VS Code Live Server
    model = await tf.loadLayersModel('http://127.0.0.1:8887/tfjs_model/model.json');
    isModelLoaded = true;
 
    //Memanggil word_index
    // const word_indexjson = await fetch('http://127.0.0.1:5500/word_index.json'); // Untuk VS Code Live Server
    const word_indexjson = await fetch('http://127.0.0.1:8887/word_index.json');
    word2index = await word_indexjson.json();
 
    console.log(model.summary());
    console.log('Model & Metadata Loaded Successfully');
}
// -----------------------------------

# **Menguji Hasil Deployment**
Pada tahap ini Anda akan menguji hasil deployment dengan menjalankan prediksi sentimen dari reviu yang sudah dimasukkan. Untuk menjalankan prediksi, bukalah aplikasi “Web Server for Chrome” dan ikuti tahapan berikut:

1. Pilihlah folder tempat Anda menyimpan proyek ini.
2. Aktifkan Web Server for Chrome dengan menggeser toggle hingga status “Web Server:STARTED”.
3. Salin http://127.0.0.1:8887 pada alamat browser, lalu buka file ‘index.html’.
Berikut merupakan contoh tampilan “Web Server for Chrome” beserta tahapan untuk mengaktifkannya.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:84e8a07cca65e48c108a0c720a0c838920211208100142.jpeg

Jika semua tahapan berjalan dengan baik, akan muncul tampilan awal dari laman web sebagai berikut.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:37fc1177b8b062537d01b626ab469ebc20211208100249.jpeg

Pada laman ini user dapat memasukkan reviu untuk restoran. Kemudian, ketika tombol Post Review ditekan, laman mengembalikan hasil berupa prediksi sentimen reviu. Hasil prediksi ini berupa klasifikasi antara reviu positif atau negatif.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:c610d8f8454b940ae8f9c1be43bbfcc420211208100352.jpeg

Selesai! 

Sampai tahap ini, Anda telah berhasil membuat proyek untuk memprediksi sentimen dari sebuah reviu restoran. Anda juga telah mampu menerapkan model pada web browser dengan TensorFlow.js serta menguji hasil deployment.

Jangan ragu untuk menerapkan model machine learning lain pada web browser dengan TensorFlow.js.